<a href="https://colab.research.google.com/github/theogbrand/learning_llm_handbook/blob/main/notebooks/Week1_LLM_Fundamentals/Week1_LLM_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1: LLM Fundamentals and Development Environment Setup
Resource Required: Google Colab (T4) GPU - free

💡 NOTE: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4.



## Notebook 1.1: LLM Tools and Libraries
- Import SEA-LION using Hugging Face, implement different decoding strategies
- Run inference on Colab T4 GPU and compare with local Ollama deployment
- Explore PyTorch basics for LLM implementation

In [ ]:
# Import required libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

## Notebook 1.2: Tokenizers Implementation
- Compare SentencePiece, TikToken, and BPE on various text samples
- Implement a basic BPE tokenizer from scratch (Karpathy Tokenizer video)
- Experiment with vocabulary extension techniques

In [ ]:
# Tokenizer implementation code

## Notebook 1.3: Embeddings Workshop
- Implement contrastive learning with CLIP for image-text pairs
- Create and visualize Word2Vec and contextual embeddings
- Evaluate embeddings using MTEB benchmark tasks
- Compare embedding performance across multiple languages

In [ ]:
# Embeddings implementation code